In [57]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [58]:
# Configuration

discovery_data_file = '../../data/Supp_Table_6_filtered_lfq_discovery.csv'

validation_data_file = '../../data/Supp_Table_3_lfq_intensities_validation.csv'
clinical_data_file = '../../data/Supp_Table_1_clinical_data.csv'


nine_prot_classifier = ['ENPP3', 'IVL', 'S100A2', 'MYH11', 'SERPINB5', 'NNMT', 'CLCA4', 'CD109', 'S100A14']
sixteen_prot_classifier = ['SERPINB5', 'S100A2', 'S100A14', 'NNMT', 'SPRR3', 'SFN', 'S100P', 'MIF', 'IVL', 'FSTL1', 'TPPP3', 'PCBD1', 'HSPA2', 'CEACAM5', 'BGN', 'ASRGL1']
ten_prot_classifier = ['SERPINB5', 'S100A2', 'S100A14', 'NNMT', 'SPRR3', 'SFN', 'S100P', 'MIF', 'IVL', 'FSTL1']
five_prot_classifier = ['SERPINB5', 'S100A2', 'S100A14', 'NNMT', 'IVL']
four_prot_classifier = ['SERPINB5', 'S100A2', 'S100A14', 'NNMT']
classifier = sixteen_prot_classifier

# Paper: K-fold corsso validation (250 mal wiederholt) 
# mit random sampling of 85 % as train and 15 % as validation
test_size = 0.3
random_state = 109

In [59]:
# Read discovery data 

gene_df = pd.read_csv(discovery_data_file, sep=';', header=0)
gene_df = gene_df.drop(columns=['Razor + unique peptides', 'Unique peptides','Q-value', 
                      'Score', 'Intensity', 'MS/MS count', 'Protein IDs',
       'Majority protein IDs', 'Protein names', 'Column1', 'Column2',
       'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8',
       'Column9', 'Column10', 'Column11', 'Column12'])
gene_df = gene_df.fillna(value="labels", limit=1)
gene_df = gene_df.set_index('Gene names')
gene_df = gene_df.transpose()
mapping = {'Healthy': 0, 'Patient': 1}
gene_df = gene_df.replace({'labels': mapping})

display(gene_df.head(5))

Gene names,A1BG,A2M,AARS,ABCE1,ABCF1,ABHD14B,ABI1,ABR,ACADVL,ACAP2,...,YWHAE,YWHAG,YWHAH,YWHAQ,YWHAZ,ZAK;pk,ZC3HAV1,ZNF185,ZYX,labels
LFQ intensity BUL_103,25.39842,26.66044,23.51749,22.38195,20.65106,23.94708,22.80967,19.92877,23.40912,20.6044,...,27.00389,24.42918,23.72848,25.59167,27.46972,21.48316,21.66006,21.59405,23.42522,0
LFQ intensity BUL_30,28.04843,29.83939,24.725,21.4001,18.52887,25.35803,19.56752,18.04529,20.12199,19.46165,...,28.6475,25.80799,24.62271,27.62819,29.19315,21.95957,19.84275,20.02877,20.17072,0
LFQ intensity BUL_40,24.33402,25.49622,23.27213,22.20973,19.87752,23.89171,22.45669,21.20985,22.55173,20.05775,...,28.13482,25.42211,25.38281,26.26785,28.2209,19.78749,22.28619,22.10188,21.40671,0
LFQ intensity BUL_47,26.15509,29.6653,18.98055,19.4373,20.06988,23.32896,21.12136,17.5265,21.38506,20.32839,...,26.75646,23.68603,24.00967,24.367,28.00657,19.62387,18.94378,21.4315,22.04613,1
LFQ intensity BUL_48,25.64636,26.88064,22.68656,21.88852,20.69693,24.13485,22.49685,20.13295,22.7297,20.86507,...,27.03444,24.14499,24.0748,25.49063,27.62415,22.0415,20.7982,22.42206,22.2678,1


In [60]:
# Read Validation Data

gene_df = pd.read_csv(validation_data_file, sep=';', header=0)
gene_df = gene_df.drop(columns=['Razor + unique peptides', 'Unique peptides','Q-value', 
                      'Score', 'Intensity', 'MS/MS count', 'Protein IDs',
       'Majority protein IDs', 'Protein names', 'Column1', 'Column2',
       'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8',
       'Column9', 'Column10', 'Column11', 'Column12', 'Column13', 'Sequence coverage [%]', 'Mol. weight [kDa]' ])
gene_df = gene_df.drop(columns=['LFQ intensity MUL_38','LFQ intensity BUL_109','LFQ intensity UL_19',
    'LFQ intensity UL_22','LFQ intensity UL_3','LFQ intensity UL_23','LFQ intensity BUL_101','LFQ intensity UL_37',
    'LFQ intensity UL_35','LFQ intensity MUL_38','LFQ intensity MUL_69'])

gene_names = list(gene_df.columns)

gene_df = gene_df.fillna(value=0)
gene_df = gene_df.set_index('Gene names')
gene_df = gene_df.transpose()


clinical_data_df = pd.read_csv('../../data/Supp_Table_1_clinical_data.csv', sep=';', header=1)
clinical_data_df = clinical_data_df[clinical_data_df["Set"] == 'Validation']
clinical_data_df['Sample ID'] = clinical_data_df['Sample ID'].str.replace('-','_')
sample_col_list = clinical_data_df['Sample ID'].tolist()
sample_col_list = [('LFQ intensity '+ sample) for sample in sample_col_list]
clinical_data_df['samples'] = sample_col_list
clinical_data_df = clinical_data_df[['samples', 'Signature prediction2']]
### 'LFQ intensity UL_37' kommt in der Tabelle 3 nicht vor -> Warum auch immer?
clinical_data_df = clinical_data_df[clinical_data_df["samples"] != 'LFQ intensity UL_37']
clinical_data_df = clinical_data_df.set_index('samples')
clinical_data_df = clinical_data_df.rename(columns={'Signature prediction2': 'labels'})

gene_df = gene_df.join(clinical_data_df)
mapping = {'Healthy': 0, 'Patient': 1}
gene_df = gene_df.replace({'labels': mapping})

#display(gene_df.head(5))

In [61]:
y = gene_df['labels']
X = gene_df[classifier]
display(X)

,SERPINB5,S100A2,S100A2,S100A14,NNMT,SPRR3,SFN,S100P,MIF,IVL,FSTL1,TPPP3,PCBD1,HSPA2,CEACAM5,BGN,ASRGL1
LFQ intensity BUL_1,26.934383,28.780222,27.951216,31.119478,26.234308,31.806511,30.314625,27.641274,30.130136,27.796001,25.411930,29.546352,25.911800,26.249866,29.586365,26.598892,28.167631
LFQ intensity BUL_10,31.157681,29.826689,32.442554,30.841986,27.504230,34.188808,31.581902,28.804091,30.797302,30.505499,23.679493,29.160292,24.213192,24.617397,31.029930,24.054407,28.553255
LFQ intensity BUL_102,30.237516,30.362648,26.164497,31.451353,25.266315,33.085327,31.550917,29.227705,29.938175,29.457148,0.000000,24.050842,24.618685,0.000000,31.052973,25.280510,0.000000
LFQ intensity BUL_11,28.497166,28.247896,0.000000,29.326696,25.257710,31.949352,32.106087,27.514425,29.079409,26.189409,26.279783,25.945299,0.000000,26.095449,26.916500,0.000000,28.174932
LFQ intensity BUL_111,31.816769,27.413771,0.000000,32.394047,27.345959,33.577454,31.594994,29.085880,29.473595,30.764538,23.887533,24.718792,24.423550,24.189108,31.352949,28.845177,27.230318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LFQ intensity UL_5,24.089340,23.113943,0.000000,25.186691,25.462763,20.517204,29.403284,26.445122,29.995234,0.000000,25.395027,27.821346,26.718819,26.153137,22.971304,26.517162,28.180445
LFQ intensity UL_53,27.156149,25.904606,0.000000,29.314646,25.237904,30.124601,29.497755,28.072193,30.718737,29.105721,24.698853,28.333643,25.470541,27.758118,29.066893,28.914875,28.879118
LFQ intensity UL_7,29.092827,27.017334,0.000000,27.633118,25.565231,30.018103,31.909100,26.034018,31.518034,26.744324,24.584801,27.199011,25.779465,25.797636,28.120356,27.753597,26.914791
LFQ intensity UL_8,0.000000,0.000000,0.000000,26.520119,28.388374,22.962063,29.922949,25.756489,31.133692,0.000000,25.529339,29.582388,26.750088,26.641685,0.000000,25.690645,29.222809


In [62]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [63]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))


from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# Model Sensitivity
print(f"Sensitivity im Paper 70% (== recall): {tp/ (tp+fn)}")

# Model Specifity
print(f"Specifity im Paper 76.2%: {tn / (tn+fp)}")

[[15 12]
 [ 6 13]]
Accuracy: 0.6086956521739131
Precision: 0.52
Recall: 0.6842105263157895
Sensitivity im Paper 70% (== recall): 0.6842105263157895
Specifity im Paper 76.2%: 0.5555555555555556


In [64]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer

specificity = make_scorer(recall_score, pos_label=0)
sensitivity = make_scorer(recall_score, pos_label=1)

clf = GaussianNB()
from sklearn.model_selection import KFold


rand = 0
stop = 1.0
prec_scores = [0.0, 0.0, 0.0, 0.0]

while( stop > 0.1):
    rand = rand +1
    kf = KFold(n_splits=3, shuffle=True, random_state=rand)
    cv = kf
    prec_scores = cross_val_score(clf, X, y, cv=cv, scoring='precision')
    stop = prec_scores.std()



# n = 3 -> rand: 3192
print(f"RAND: {rand}, stop: {stop}")
kf = KFold(n_splits=3, shuffle=True, random_state=3191)
cv = kf

acc_scores = cross_val_score(clf, X, y, cv=cv)
print(acc_scores)
print("%0.2f (std %0.2f)" % (acc_scores.mean(), acc_scores.std()))

prec_scores = cross_val_score(clf, X, y, cv=cv, scoring='precision')
print(prec_scores)
print("%0.2f (std %0.2f)" % (prec_scores.mean(), prec_scores.std()))

specifity_scores = cross_val_score(clf, X, y, cv=cv, scoring=specificity)
print(specifity_scores)
print("%0.2f (std %0.2f)" % (specifity_scores.mean(), specifity_scores.std()))

sensitivity_scores = cross_val_score(clf, X, y, cv=cv, scoring=sensitivity)
print(sensitivity_scores)
print("%0.2f (std %0.2f)" % (sensitivity_scores.mean(), sensitivity_scores.std()))

RAND: 1, stop: 0.0741131272784796
[0.62745098 0.76       0.56      ]
0.65 (std 0.08)
[0.38888889 0.65217391 0.45833333]
0.50 (std 0.11)
[0.69444444 0.74193548 0.56666667]
0.67 (std 0.07)
[0.46666667 0.78947368 0.55      ]
0.60 (std 0.14)
